In [2]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas matplotlib scipy cartopy openmeteo_requests retry_requests fipy

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import openmeteo_requests
import requests_cache
from retry_requests import retry
from openmeteo_sdk.Variable import Variable

In [3]:
#https://github.com/open-meteo/python-requests/tree/main#readme

In [8]:
def extract_data_from_resp(response, wind_data, nb_coord): #Sort un dataframe avec les valeurs pour les 4 fuseaux horaires des nb_coord coordonnées
    for i in range(nb_coord): #On prend les données coordonnée par coordonnée
        resp = response[i] #i-ème coordonnée 
        hourl = resp.Hourly()
        
        hourl_wind_speed_10m = hourl.Variables(0).ValuesAsNumpy()
        hourl_wind_speed_100m = hourl.Variables(1).ValuesAsNumpy()
        hourl_wind_direction_10m = hourl.Variables(2).ValuesAsNumpy()
        hourl_wind_direction_100m = hourl.Variables(3).ValuesAsNumpy()
        hourl_data = {"date": pd.date_range(
            start = pd.to_datetime(hourl.Time(), unit = "s", utc = True),
            end = pd.to_datetime(hourl.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = hourl.Interval()),
            inclusive = "left"
        )}
        
        hourl_data["spd10"] = hourl_wind_speed_10m
        hourl_data["dir10"] = hourl_wind_direction_10m
        hourl_data["spd100"] = hourl_wind_speed_100m
        hourl_data["dir100"] = hourl_wind_direction_100m
    
        hourl_df = pd.DataFrame(data = hourl_data)
        #print(hourl_df)
        wind_data.at[i, "date"] = hourl_data["date"][0]
        wind_data.at[i, "spd10"] = hourl_data["spd10"][0]
        wind_data.at[i, "dir10"] = hourl_data["dir10"][0]
        wind_data.at[i, "spd100"] = hourl_data["spd100"][0]
        wind_data.at[i, "dir100"] = hourl_data["dir100"][0]
        #print(wind_data)
    return wind_data

In [18]:
def extract_data_from_api(lat_min, lat_max, lon_min, lon_max, pas, start_d, end_d):
    #dates au format "AAAA-MM-JJ"
                          
    latitudes = np.arange(lat_min, lat_max, pas)
    longitudes = np.arange(lon_min, lon_max, pas)
    
    latitudes_list = latitudes.round(3).tolist()
    longitudes_list = longitudes.round(3).tolist()
    
    l_latitudes = np.repeat(latitudes_list, len(longitudes)).tolist()
    l_longitudes = np.tile(longitudes_list, len(latitudes)).tolist()
    #print(l_latitudes, l_longitudes)
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
    	"latitude": l_latitudes,
    	"longitude": l_longitudes,
    	"start_date": start_d,
    	"end_date": end_d,
    	"hourly": ["wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m"],
        "temporal_resolution": "hourly_6"
    } #On demande les infos toutes les 6h. Cela renvoie donc 16x4 = 64 valeurs par jour
    responses = openmeteo.weather_api(url, params=params)
    
    wind_data = pd.DataFrame({'lat': l_latitudes, 'lon': l_longitudes, 'date' : '2024-09-09 00:00:00+00:00' ,'spd10': 0.0, 'spd100': 0.0,'dir10':0.0, 'dir100':0.0})

    extract_data_from_resp(responses, wind_data, len(l_latitudes))
    
    return wind_data

In [19]:
#Initialisation des variables
lat_min = 41.5
lat_max = 42.21 #43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.06
start_d = "2024-09-09"
end_d = "2024-09-09"

In [20]:
df = extract_data_from_api(lat_min, lat_max, lon_min, lon_max, pas, start_d, end_d)
 

In [257]:
df_06_422_43 

In [260]:
df_06_415_422 = df
"""#Initialisation des variables
lat_min = 41.5
lat_max = 42.21 #43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.06
start_d = "2024-09-09"
end_d = "2024-09-09"""

'#Initialisation des variables\nlat_min = 41.5\nlat_max = 42.21 #43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.06\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [264]:
df_442 = pd.DataFrame(np.concat([df_06_415_422,df_06_422_43]))

In [253]:
df_415_422 
"""#Initialisation des variables
lat_min = 42.21
lat_max = 43.01 #43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.07
start_d = "2024-09-09"
end_d = "2024-09-09"""

'#Initialisation des variables\nlat_min = 42.21\nlat_max = 43.01 #43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.07\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [242]:
df_422_43 = df

In [226]:
df_425_43 = df

In [244]:
print(len(df_422_43))

180


In [250]:
df_345 = pd.DataFrame(np.concat([df_415_422,df_422_43]))

In [252]:
df_693
"""#Initialisation des variables
lat_min = 42.21
lat_max = 43.01 #43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.05
start_d = "2024-09-09"
end_d = "2024-09-09"""

'#Initialisation des variables\nlat_min = 42.21\nlat_max = 43.01 #43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.05\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [151]:
df_176 
"""
lat_min = 41.5
lat_max = 43.02
lon_min = 8.5
lon_max = 9.52
pas = 0.1"""

In [156]:
print(len(df))

176


In [161]:
df_204 
"""
lat_min = 41.5
lat_max = 43.02
lon_min = 8.5
lon_max = 9.52
pas = 0.09
start_d = "2024-09-09"
end_d = "2024-09-09"""

'\nlat_min = 41.5\nlat_max = 43.02\nlon_min = 8.5\nlon_max = 9.52\npas = 0.09\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [166]:
df_247 
"""
#Initialisation des variables
lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.08
start_d = "2024-09-09"
end_d = "2024-09-09"
"""

'\n#Initialisation des variables\nlat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.08\nstart_d = "2024-09-09"\nend_d = "2024-09-09"\n'

In [178]:
df_260
"""
lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.078
start_d = "2024-09-09"
end_d = "2024-09-09"""

'\nlat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.078\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [181]:
len(df)

117

In [182]:
df_117 
"""lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.12
start_d = "2024-09-09"
end_d = "2024-09-09"""

'lat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.12\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [186]:
print(len(df))

77


In [188]:
df_77
"""#Initialisation des variables
lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.15
start_d = "2024-09-09"
end_d = "2024-09-09"""

'#Initialisation des variables\nlat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.15\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [191]:
len(df)

48

In [192]:
df_48
"""
#Initialisation des variables
lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.2
start_d = "2024-09-09"
end_d = "2024-09-09"""

'\n#Initialisation des variables\nlat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.2\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [195]:
len(df)

54

In [196]:
df_54 
"""#Initialisation des variables
lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.2
start_d = "2024-09-09"
end_d = "2024-09-09"""

'#Initialisation des variables\nlat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.2\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [207]:
len(df)

260

In [209]:
df_260 
"""#Initialisation des variables
lat_min = 41.5
lat_max = 43.01
lon_min = 8.5
lon_max = 9.51
pas = 0.0778
start_d = "2024-09-09"
end_d = "2024-09-09"""

'#Initialisation des variables\nlat_min = 41.5\nlat_max = 43.01\nlon_min = 8.5\nlon_max = 9.51\npas = 0.0778\nstart_d = "2024-09-09"\nend_d = "2024-09-09'

In [265]:
df_48.to_csv('../data/API/df_48.csv', index = False)
df_54.to_csv('../data/API/df_54.csv', index = False)
df_77.to_csv('../data/API/df_77.csv', index = False)
df_117.to_csv('../data/API/df_117.csv', index = False)
df_176.to_csv('../data/API/df_176.csv', index = False)
df_204.to_csv('../data/API/df_204.csv', index = False)
df_247.to_csv('../data/API/df_247.csv', index = False)
df_260.to_csv('../data/API/df_260.csv', index = False)
df_345.to_csv('../data/API/df_345.csv', index = False)
df_442.to_csv('../data/API/df_442.csv', index = False)
df_693.to_csv('../data/API/df_693.csv', index=False)